## Programming for Computations - A Gentle Introduction to Numerical Simulations with Python

Notes on: https://hplgit.github.io/Programming-for-Computations/pub/p4c/p4c_Python.pdf

In [5]:
import sys 
sys.path.insert(1,'../')
import utils.utils as utils

import numpy as np
import sympy as sym
import plotly.graph_objects as go
import plotly.express as px

### 3.4.2 Proper test procedures
There are three serious ways to test the implementation of numerical methods via unit tests:
1. Comparing with hand-computed results in a problem with few arithmetic operations, i.e., small n.
2. Solving a problem without numerical errors. We know that the trapezoidal rule must be exact for linear functions. The error produced by the program must then be zero (to machine precision).
3. Demonstrating correct convergence rates. A strong test when we can compute exact errors, is to see how fast the error goes to zero as n grows. In the trapezoidal and midpoint rules it is known that the error depends on n as n^−2 as n→∞.


Numerical integration methods usually have an error that converge to zero as n^−p for some p that depends on the method.

In [6]:
a, b = 0.1, 0.2
expected = 0.3
a + b == expected

False

In [7]:
tol = 1E-15
diff = abs(a + b - expected)
diff < tol

True

In [8]:
diff

5.551115123125783e-17

In [9]:
print('k    Error (~E-(16-k))')
for k in range(10):
    print(f'{k}    {10**k + 0.3 - 0.1 - (10**k +0.1 + 0.2 - 0.1): 1.15g}')

k    Error (~E-(16-k))
0     0
1     1.77635683940025e-15
2     0
3    -1.13686837721616e-13
4    -1.81898940354586e-12
5     0
6     1.16415321826935e-10
7     1.86264514923096e-09
8     0
9    -1.19209289550781e-07


This means that the tolerance must be larger if we compute with larger numbers. Setting a proper tolerance therefore requires some experiments to see what level of accuracy one can expect. A way out of this difficulty is to work with **relative instead of absolute differences**. In a relative difference we divide by one of the operands, e.g.:

In [10]:
print('k    Error (~E-16)')
a = 10**k + 0.3
b = 10**k + 0.1 + 0.2
for k in range(10):
    print(f'{k}    {(a-b)/a: 1.15g}')

k    Error (~E-16)
0    -1.19209289515018e-16
1    -1.19209289515018e-16
2    -1.19209289515018e-16
3    -1.19209289515018e-16
4    -1.19209289515018e-16
5    -1.19209289515018e-16
6    -1.19209289515018e-16
7    -1.19209289515018e-16
8    -1.19209289515018e-16
9    -1.19209289515018e-16


The requirements to a test function are simple:
- the name must start with test_
- the test function cannot have any arguments
- the tests inside test functions must be boolean expressions
- a boolean expression b must be tested with assert b, msg, where
msg is an optional object (string or number) to be written out when b is false

In [11]:
def add(a, b):
    return a + b

def test_add():
    expected = 1 + 1
    computed = add(1, 1)
    assert computed == expected, f'1+1={computed: .15g}' 

def trapezoidal(f, a, b, n):
    h = float(b-a)/n
    result = 0.5*f(a) + 0.5*f(b)
    for i in range(1, n):
        result += f(a + i*h)
    result *= h
    return result

Running py.test -s -v will run all tests in all test*.py files in all subdirectories

Hand-computed numerical results

In [12]:
def test_trapezoidal_one_exact_result():
    """Compare one hand-computed result."""
    from math import exp
    v = lambda t: 3*(t**2)*exp(t**3)
    n=2
    computed = trapezoidal(v, 0, 1, n) 
    expected = 2.463642041244344
    error = abs(expected - computed)
    tol = 1E-14
    success = error < tol
    msg = f'error={error} > tol={tol}' 
    assert success, msg

Solving a problem without numerical errors

In [13]:
def test_trapezoidal_linear():
    """Check that linear functions are integrated exactly."""
    f = lambda x: 6*x - 4
    F = lambda x: 3*x**2 - 4*x  # Anti-derivative
    a = 1.2
    b = 4.4
    expected = F(b) - F(a)
    tol = 1E-14
    for n in 2, 20, 21:
        computed = trapezoidal(f, a, b, n)
        error = abs(expected - computed)
        success = error < tol
        msg = f'n={n}, err={error}'
        assert success, msg

Demonstrating correct convergence rates

Computing convergence rates requires somewhat more tedious pro- gramming than the previous tests, but can be applied to more general integrands. The algorithm typically goes like
• for i = 0,1,2,...,q
– ni = 2i+1
– Compute integral with ni intervals – Compute the error Ei
– Estimate ri from (3.24) if i > 0

In [15]:
def convergence_rates(f, F, a, b, num_experiments=14):
    from math import log
    from numpy import zeros
    expected = F(b) - F(a)
    n = zeros(num_experiments, dtype=int)
    E = zeros(num_experiments)
    r = zeros(num_experiments-1)
    for i in range(num_experiments):
        n[i] = 2**(i+1)
        computed = trapezoidal(f, a, b, n[i])
        E[i] = abs(expected - computed)
        if i > 0:
            r_im1 = log(E[i]/E[i-1])/log(float(n[i])/n[i-1])
        r[i-1] = float(f'{r_im1:.2f}') # Truncate to two decimals
    return r